In [20]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
#from teamName import getMyPosition 

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000


def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T


pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)
print("Loaded %d instruments for %d days" % (nInst, nt))




Loaded 50 instruments for 500 days


#### EMA Calculating Function

In [22]:
# Function to calculate EMA
def calculate_ema(prices, period):
    alpha = 2 / (period + 1)
    ema = np.zeros_like(prices)
    ema[0] = prices[0]  # Initialize with the first price
    for t in range(1, len(prices)):
        ema[t] = alpha * prices[t] + (1 - alpha) * ema[t-1]
    return ema

#### MACD Trading Strategy installed into getMyPosition Function

#### Optimise the days of shorter and longer EMA so that trading signals

### Check my code inside getMyPosition whether is written correctly or not

In [24]:
##### TODO #########################################
### RENAME THIS FILE TO YOUR TEAM NAME #############
### IMPLEMENT 'getMyPosition' FUNCTION #############
### TO RUN, RUN 'eval.py' ##########################

nInst = 50
currentPos = np.zeros(nInst)

def getMyPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape
    if nt < 26:
        return np.zeros(nins)  # Not enough data points to calculate EMAs
    
    # Initialize positions
    new_positions = np.zeros(nins)
    # Iterate over each instrument
    for i in range(nins):
        prices = prcSoFar[i, :].copy()

        # Calculate 12-period and 26-period EMAs
        ema_12 = calculate_ema(prices, 12)   ### optimise the day here
        ema_26 = calculate_ema(prices, 26)   ### optimise the day here
    
        # Current price of the stock
        current_price = prices[-1].copy()
        
        # Generate trading signals based on EMA crossover
        if ema_12[-1] > ema_26[-1] and ema_12[-2] <= ema_26[-2]:
            # Buy signal: 12-period EMA crosses above 26-period EMA
            # Calculate the number of shares to buy without exceeding $10,000
            num_shares_to_buy = 10000 // current_price
            new_positions[i] = num_shares_to_buy
        elif ema_12[-1] < ema_26[-1] and ema_12[-2] >= ema_26[-2]:
            # Sell signal: 12-period EMA crosses below 26-period EMA
            # Calculate the number of shares to sell (short) without exceeding $10,000
            num_shares_to_sell = -10000 // current_price
            new_positions[i] = num_shares_to_sell
            
        else:
            # No signal, maintain current position
            if abs(currentPos[i]*current_price) > 10000:
                currentPos[i] = np.sign(currentPos[i])*10000 // current_price
            else:
                new_positions[i] = currentPos[i] 
    # Update current positions
    
    currentPos = np.array(new_positions)
    return currentPos


In [25]:
print(np.sign(5))

1


#### P&L Function  optimise the trading at line for t in range(50,501)

In [27]:
def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(50, 501):   ### day start to trade at 50 to 500
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        ### this is my modification code for checking position
        #print(newPosOrig) ###check position Number of holding shares
        print(curPrices)
        print(newPos)   ### position number of holding shares after apply limited constraints
        print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
              (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)

#### Calculating P&L by calling calcPL function with stock price input

In [29]:
(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)

[13.07 72.5  48.19 50.58 54.68 12.32 18.33 46.6  68.51 50.44 38.48 29.57
 27.55 46.75 15.83 22.76 35.94 45.13 14.41 30.37 55.49 22.81 65.34 31.7
 63.29 42.77 54.76 29.65 51.01 31.54 13.83 63.75 55.23 47.36 27.92 52.75
 32.64 37.83 21.7  49.32 32.54 56.25 13.9  63.68 39.05 52.33 54.73 35.41
 43.47 57.12]
[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0. -153.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.]
Day 50 value: -10.00 todayPL: $-10.00 $-traded: 9997 return: -0.00100
[13.1  72.49 48.17 50.72 54.45 12.33 18.35 46.85 68.62 49.64 38.42 29.77
 27.5  47.03 15.51 22.81 35.95 45.13 14.46 30.61 55.87 22.61 64.54 31.76
 63.35 43.94 55.28 29.64 50.96 31.51 13.72 63.76 54.91 47.62 27.84 53.21
 32.69 37.85 21.68 49.28 32.37 56.72 13.91 63.05 39.06 52.46 55.02 35.09
 43.47 57.23]
[   0.    0. 

In [30]:
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)

=====
mean(PL): -32.4
return: -0.00097
StdDev(PL): 175.73
annSharpe(PL): -2.91 
totDvolume: 15098660 
Score: -49.96
